<a href="https://colab.research.google.com/github/WasikulIslam1/CIS-4400-Project-9/blob/main/Location_ETL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install azure-storage-blob
!pip install pyarrow

In [5]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient



Use of the free database in production requires that you link back to:
https://simplemaps.com/data/us-counties.

In [103]:

# Specify the file path of your CSV file
file_path = "uscounties.csv"
file_path2 ="zipdata.csv"
# Read the CSV file into a DataFrame
df2 =pd.read_csv(file_path2)
df = pd.read_csv(file_path)

# df.drop(columns=['population','county','county_ascii'], inplace=True)
# df.head()
df2 = pd.read_csv(file_path2, skiprows=[0])

# Set the column names manually
column_names = ["Zip Code", "City name", "USPS State Code", "State Name",
                "ZCTA", "ZCTA parent", "Pop", "Density", "County_Code",
                "County_Name", "County Weights", "CountyName",
                "Codes", "Imprecise", "Military Timezone", "Geo Point","latlong"]
df2.columns = column_names

# Merge the two DataFrames on the specified columns
merged_df = pd.merge(df, df2, left_on=['county_fips', 'county'], right_on=['County_Code', 'County_Name'], how='inner')
merged_df.tail(3)

<ipython-input-103-aeb2d48fd821>:5: DtypeWarning: Columns (0,4,5,6,7,8,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 =pd.read_csv(file_path2)


,county,county_ascii,county_full,county_fips,state_id,state_name,lat,lng,population,Zip Code,...,Density,County_Code,County_Name,County Weights,CountyName,Codes,Imprecise,Military Timezone,Geo Point,latlong
32673,Kenedy,Kenedy,Kenedy County,48261,TX,Texas,26.9285,-97.7017,116,78338,...,0.3,48261,Kenedy,"{""48261"": ""100""}",Kenedy,48261,False,False,America/Chicago,"26.89255, -97.76255"
32674,Loving,Loving,Loving County,48301,TX,Texas,31.8493,-103.5800,96,79754,...,0.2,48301,Loving,"{""48301"": ""100""}",Loving,48301,False,False,America/Chicago,"31.80166, -103.59994"
32675,Kalawao,Kalawao,Kalawao County,15005,HI,Hawaii,21.1709,-156.9475,50,96742,...,2.1,15005,Kalawao,"{""15005"": ""100""}",Kalawao,15005,False,False,Pacific/Honolulu,"21.17087, -156.94743"


In [108]:

merged_df["location_id"] = range(1, len(merged_df) + 1)
merged_df = merged_df.rename(columns={
    "county_fips": "county_fips",
    "county": "county_name",
    "Zip Code":"zipcode",
    "lat": "latitude",
    "lng":"longitude",
    "state_name":"state_name",
    "state_id":"state_id"})
new_order = ["location_id",
             "county_fips",
             "county_name",
             "state_id",
             "state_name",
             "latitude",
             "longitude",
             ]

dim_location = merged_df[new_order]

dim_location

,location_id,county_fips,county_name,state_id,state_name,latitude,longitude
0,1,6037,Los Angeles,CA,California,34.3219,-118.2247
1,2,6037,Los Angeles,CA,California,34.3219,-118.2247
2,3,6037,Los Angeles,CA,California,34.3219,-118.2247
3,4,6037,Los Angeles,CA,California,34.3219,-118.2247
4,5,6037,Los Angeles,CA,California,34.3219,-118.2247
...,...,...,...,...,...,...,...
32671,32672,48269,King,TX,Texas,33.6165,-100.2558
32672,32673,48261,Kenedy,TX,Texas,26.9285,-97.7017
32673,32674,48261,Kenedy,TX,Texas,26.9285,-97.7017
32674,32675,48301,Loving,TX,Texas,31.8493,-103.5800


In [107]:
import os
dim_location.to_csv('dim_location1.csv', index=False)


In [96]:
# Load Azure Storage connection string and container name from config.json
with open('config.json', 'r') as f:
    config = json.load(f)

connection_string = config['connectionString']


# Path to your local CSV file
container_name = 'location'
file_path = "dim_location.csv"
blob_name = "location_cleaned"

# Create a blob service client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Create a blob client using the container and blob name
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload the file
with open(file_path, "rb") as data:
    blob_client.upload_blob(data)